In [62]:
import pandas as pd
import numpy as np

* composite key
* different events and their counts as a score

In [63]:
data = {'organization_id': [1, 1, 0, 2, 3, 3, 4, 0, 4, 4],
        'customer_id': [1, 1, 1, 2, 3, 3, 4, 4, 4, 4],
        'summary_day': ['2015-04-20'] * 10,
        'event': ['add_t', 'add_t', 'drop_f', 'drop_f', 'drop_t', 'add_f', 'add_f', 'add_t', 'add_t', 'drop_t'],
        'score': [1] * 10,}

## groupby

In [64]:
class grp():
    df = None

In [65]:
grp.df = pd.DataFrame(data)

groups = grp.df.groupby(['summary_day', 'customer_id', 'organization_id', 'event'])
r = groups.aggregate(np.sum)
r

score
summary_day customer_id organization_id event        
2015-04-20  1           0               drop_f      1
                        1               add_t       2
            2           2               drop_f      1
            3           3               add_f       1
                                        drop_t      1
            4           0               add_t       1
                        4               add_f       1
                                        add_t       1
                                        drop_t      1

In [66]:
r.reset_index(inplace=True)
r

,summary_day,customer_id,organization_id,event,score
0,2015-04-20,1,0,drop_f,1
1,2015-04-20,1,1,add_t,2
2,2015-04-20,2,2,drop_f,1
3,2015-04-20,3,3,add_f,1
4,2015-04-20,3,3,drop_t,1
5,2015-04-20,4,0,add_t,1
6,2015-04-20,4,4,add_f,1
7,2015-04-20,4,4,add_t,1
8,2015-04-20,4,4,drop_t,1


## pivot

In [67]:
df = pd.DataFrame(data)

### combine composite key into tuple

In [68]:
df['uniq'] = zip(df['customer_id'], df['organization_id'], df['summary_day'], df.index.values)

print ">>> uniq tuple added <<<"
print df.uniq.head(n=10)

>>> uniq tuple added <<<
0    (1, 1, 2015-04-20, 0)
1    (1, 1, 2015-04-20, 1)
2    (1, 0, 2015-04-20, 2)
3    (2, 2, 2015-04-20, 3)
4    (3, 3, 2015-04-20, 4)
5    (3, 3, 2015-04-20, 5)
6    (4, 4, 2015-04-20, 6)
7    (4, 0, 2015-04-20, 7)
8    (4, 4, 2015-04-20, 8)
9    (4, 4, 2015-04-20, 9)
Name: uniq, dtype: object


In [69]:
df = df.pivot('uniq', 'event', 'score')
df.fillna(0, inplace=True)
df

event,add_f,add_t,drop_f,drop_t
uniq,,,,
"(1, 0, 2015-04-20, 2)",0,0,1,0
"(1, 1, 2015-04-20, 0)",0,1,0,0
"(1, 1, 2015-04-20, 1)",0,1,0,0
"(2, 2, 2015-04-20, 3)",0,0,1,0
"(3, 3, 2015-04-20, 4)",0,0,0,1
"(3, 3, 2015-04-20, 5)",1,0,0,0
"(4, 0, 2015-04-20, 7)",0,1,0,0
"(4, 4, 2015-04-20, 6)",1,0,0,0
"(4, 4, 2015-04-20, 8)",0,1,0,0


* extract two columns from dataframe as a numpy ndarray
* add the columns (sum over the rows, which is axis=1

In [100]:
s = df[['add_t', 'add_f']].values
s

array([[ 0.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  0.]])

In [102]:
df['add_total'] = s.sum(axis=1)

In [103]:
df['add_total'].head(n=10)

uniq
(1, 0, 2015-04-20, 2)    0
(1, 1, 2015-04-20, 0)    1
(1, 1, 2015-04-20, 1)    1
(2, 2, 2015-04-20, 3)    0
(3, 3, 2015-04-20, 4)    0
(3, 3, 2015-04-20, 5)    1
(4, 0, 2015-04-20, 7)    1
(4, 4, 2015-04-20, 6)    1
(4, 4, 2015-04-20, 8)    1
(4, 4, 2015-04-20, 9)    0
Name: add_total, dtype: float64

In [74]:
def calc_sn_total(df):
    sn_add = [c for c in df.columns.tolist()
                if c.startswith('add')]
    sn_drop = [c for c in df.columns.tolist()
                if c.startswith('drop')]

    def calc(x):
        sum = 0
        for c in sn_add:
            sum += x[c]
        for c in sn_drop:
            sum -= x[c]
        return sum

    return calc


def assign_calc_columns(df):
    return df.assign(sn_total = calc_sn_total(df))

In [75]:
df = assign_calc_columns(df)

columns_score = df.columns.tolist()
columns_key = ['summary_day', 'customer_id', 'organization_id']

df['customer_id'] = [v[0] for v in df.index.values]
df['organization_id'] = [v[1] for v in df.index.values]
df['summary_day'] = [v[2] for v in df.index.values]

print ">>> add key columns back <<<"
print df.head(n=10)

>>> add key columns back <<<
event                  add_f  add_t  drop_f  drop_t  add_total  sn_total  \
uniq                                                                       
(1, 0, 2015-04-20, 2)      0      0       1       0          0        -1   
(1, 1, 2015-04-20, 0)      0      1       0       0          1         2   
(1, 1, 2015-04-20, 1)      0      1       0       0          1         2   
(2, 2, 2015-04-20, 3)      0      0       1       0          0        -1   
(3, 3, 2015-04-20, 4)      0      0       0       1          0        -1   
(3, 3, 2015-04-20, 5)      1      0       0       0          1         2   
(4, 0, 2015-04-20, 7)      0      1       0       0          1         2   
(4, 4, 2015-04-20, 6)      1      0       0       0          1         2   
(4, 4, 2015-04-20, 8)      0      1       0       0          1         2   
(4, 4, 2015-04-20, 9)      0      0       0       1          0        -1   

event                  customer_id  organization_id summar

* write the result to a csv file

In [76]:
columns_csv = columns_key + sorted(columns_score)

print ">>> csv columns <<<"
print columns_csv

df.to_csv("tmp.csv", columns=columns_csv, index=False)

>>> csv columns <<<
['summary_day', 'customer_id', 'organization_id', 'add_f', 'add_t', 'add_total', 'drop_f', 'drop_t', 'sn_total']


### split data frame and use index

In [77]:
class WorkspaceSplit():
    df = None
    df_pivot = None

spl = WorkspaceSplit()

In [78]:
spl.df = pd.DataFrame(data)

pk = spl.df[['summary_day', 'customer_id', 'organization_id']]
col = spl.df[['event', 'score']]

col.index.name = 'pix'
col.reset_index(inplace=True)

col_pivot = col.pivot('pix', 'event', 'score')
col_pivot.fillna(0, inplace=True)

spl.df_pivot =  pd.concat([col_pivot, pk], axis=1, join='inner')
spl.df_pivot

event,add_f,add_t,drop_f,drop_t,summary_day,customer_id,organization_id
pix,,,,,,,
0,0,1,0,0,2015-04-20,1,1
1,0,1,0,0,2015-04-20,1,1
2,0,0,1,0,2015-04-20,1,0
3,0,0,1,0,2015-04-20,2,2
4,0,0,0,1,2015-04-20,3,3
5,1,0,0,0,2015-04-20,3,3
6,1,0,0,0,2015-04-20,4,4
7,0,1,0,0,2015-04-20,4,0
8,0,1,0,0,2015-04-20,4,4


## Blaze

In [79]:
from blaze import by

In [80]:
class WorkspaceBlaze(object):
    df = None

blz = WorkspaceBlaze()

In [81]:
a = [1,1]
b = [2,2]
c = [4,2]
d = [0,3]

m = np.array([a, b, c, d])
print m.sum(axis=0)

cast = pd.DataFrame([2.0, np.nan])
cast.fillna(-1, inplace=True)

print cast
#print cast.apply(lambda x: str(int(x)) )

[7 8]
   0
0  2
1 -1


In [82]:
blz.df = pd.DataFrame(data)
t = Data(blz.df)

by([t.customer_id, t.organization_id], total = t.score.sum())

NotImplementedError: Could not find signature for by: <list>